# Camera

In [146]:
import pandas as pd
import nltk
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

In [140]:
%run ../funcoes.py

Loading functions...


In [2]:
domain = "camera"
dfCameraTest = pd.read_csv('../07_Keras/data/domain_camera_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfCameraTrain = pd.read_csv('../07_Keras/data/domain_camera_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

In [129]:
# carregando tipos
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tipos.loc[tipos['domain'] == 'camera']

In [31]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [20]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

In [3]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [5]:
!cat ../07_Keras/data/domain_camera_IOB_train.csv | grep Sentence | wc -l

192


In [6]:
!cat ../07_Keras/data/domain_camera_IOB_test.csv | grep Sentence | wc -l

82


In [7]:
contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

In [8]:
contTrain = round(contSents * 0.9)
print(contTrain)

172


In [9]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}
sent = []
tags = []
contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

In [10]:
len(training['sentences'])

172

In [11]:
len(validation['sentences'])

19

In [12]:
test = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfCameraTest.iterrows():
    if (row[0] != "" and count > 0):
        test['sentences'].append(sent)
        test['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
test['sentences'].append(sent)
test['tags'].append(tags)

In [13]:
len(test['sentences'])

81

## Bert Multilingual

In [14]:
transformer = 'bert-base-multilingual-uncased'

In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [16]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

2021-12-20 15:26:46.337244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 15:26:46.337278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
model.train()


 Epoch 1 / 40


 ... (more hidden) ...
 ... (more hidden) ...


Train Loss = 1.005359445299421 Valid Loss = 0.6881784399350485

 Epoch 2 / 40


 ... (more hidden) ...
 ... (more hidden) ...


Train Loss = 0.511123100561755 Valid Loss = 0.37879177927970886

 Epoch 3 / 40


...

 ... (more hidden) ...
 ... (more hidden) ...


Train Loss = 0.0032929643847247852 Valid Loss = 0.4793768525123596

 Epoch 40 / 40


 ... (more hidden) ...
 ... (more hidden) ...


Train Loss = 0.0030870044055128737 Valid Loss = 0.4793768525123596


'Model trained successfully'

In [18]:
model.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 25 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 25 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/douto

,Level,F1-Score,Precision,Recall
0,B-asp,0.343137,0.304348,0.393258
1,I-asp,0.194444,0.233333,0.166667
0,AVG_MICRO,0.304348,NaN,NaN
0,AVG_MICRO,0.268791,NaN,NaN


In [ ]:
y_pred = model.predict(test['sentences'])

In [141]:
aspectsCamera = result2aspecList(y_pred, test['sentences'])
resCamera = calculaMetricas(
    aspectsCamera, 
    aspectsTest['datasets']['camera'], 
    tipos,
    lemma=False, 
    output=False
)
resCamera

{'totalAspects': 104,
 'pred': 69,
 'tp': 55,
 'fp': 41,
 'fn': 49,
 'precisao': 0.5729166666666666,
 'precisaoLaplace': 0.5714285714285714,
 'revocacao': 0.5288461538461539,
 'medidaf': 0.5499999999999999,
 'totalExplicitos': 91,
 'explicitosOk': 25,
 'pExplicitos': 0.27472527472527475,
 'explicitsList': ['menu',
  'fotografia',
  'resolução',
  'bateria',
  'cartão de memória',
  'preço',
  'botão',
  'máquina',
  'câmera',
  'manual de instruções',
  'investimento',
  'redução de olhos vermelhos',
  'custo',
  'fotos',
  'processador',
  'camera',
  'manuseio',
  'recursos',
  'flash',
  'zoom',
  'imagens',
  'lente',
  'produto',
  'vídeo',
  'foco'],
 'totalImplicitos': 13,
 'implicitosOk': 3,
 'pImplicitos': 0.23076923076923078,
 'implicitsList': ['compacta', 'fácil de usar', 'facil de usar'],
 'tipos': {'Qualification': {'Adjective': 4}, 'Event': {'Verb': 2}}}

In [148]:
xml = dicttoxml(resCamera, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    

## Bert Portuguese: Bertimbau

In [151]:
transformer = 'neuralmind/bert-base-portuguese-cased'

In [152]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [153]:
from NERDA.models import NERDA
model2 = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [154]:
model2.train()


 Epoch 1 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.08s/it]


Train Loss = 1.0234091494764601 Valid Loss = 0.8536535898844401

 Epoch 2 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.14s/it]


Train Loss = 0.5410053953528404 Valid Loss = 0.3945716122786204

 Epoch 3 / 40


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  2.00s/it]


Train Loss = 0.0002985817929064589 Valid Loss = 0.5508960286776224

 Epoch 40 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.96s/it]

Train Loss = 0.0004517896013567224 Valid Loss = 0.5508960286776224


'Model trained successfully'

In [155]:
model2.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 25 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 25 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/douto

,Level,F1-Score,Precision,Recall
0,B-asp,0.414097,0.34058,0.528090
1,I-asp,0.189189,0.21875,0.166667
0,AVG_MICRO,0.358804,NaN,NaN
0,AVG_MICRO,0.301643,NaN,NaN


In [156]:
y_pred = model2.predict(test['sentences'])

In [157]:
aspectsCamera = result2aspecList(y_pred, test['sentences'])
resCamera = calculaMetricas(
    aspectsCamera, 
    aspectsTest['datasets']['camera'], 
    tipos,
    lemma=False, 
    output=False
)
resCamera

{'totalAspects': 104,
 'pred': 81,
 'tp': 63,
 'fp': 50,
 'fn': 41,
 'precisao': 0.5575221238938053,
 'precisaoLaplace': 0.5565217391304348,
 'revocacao': 0.6057692307692307,
 'medidaf': 0.5806451612903226,
 'totalExplicitos': 91,
 'explicitosOk': 29,
 'pExplicitos': 0.31868131868131866,
 'explicitsList': ['qualidade das fotos',
  'câmera',
  'camera',
  'armazenamento',
  'zoom',
  'vídeo',
  'produto',
  'resolução',
  'bateria',
  'ultrazoom',
  'investimento',
  'custo',
  'fotos',
  'imagens',
  'lente',
  'qualidade de imagem',
  'memória',
  'menu',
  'máquina',
  'recursos',
  'visor',
  'flash',
  'foco',
  'video',
  'fotografia',
  'funções',
  'preço',
  'botão',
  'manuseio'],
 'totalImplicitos': 13,
 'implicitosOk': 2,
 'pImplicitos': 0.15384615384615385,
 'implicitsList': ['medidas', 'compacta'],
 'tipos': {'Feature': {'Attribute': 1}, 'Qualification': {'Adjective': 4}}}

In [158]:
xml = dicttoxml(resCamera, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    